In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import Dependencies 
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
import pickle

## Preprocess

In [3]:
# Read in files

file_path = 'Data/Cleaned_Data/'

In [4]:
# Read in Health Ranking Data
df = pd.read_csv(f"../../{file_path}County_Health_All_Factors.csv")
print(df.shape)
df.head()

(3142, 10)


,Unnamed: 0,FIPS,State,County,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank
0,1,1001,Alabama,Autauga,8,5,15,14,5,50
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62
2,3,1005,Alabama,Barbour,14,48,53,30,61,32
3,4,1007,Alabama,Bibb,47,24,38,16,38,31
4,5,1009,Alabama,Blount,36,14,10,41,15,53


In [5]:
# Read in the Merged Data
df2 = pd.read_csv(f"../../{file_path}Merged_Demographic_Data.csv")
print(df2.shape)
df2.head()

(3033, 25)


,County,State,FIPS,Lyme_Disease_Incidence_Reported,Population,%<18_Yrs_Old,%65_Yrs_Old_and_over,Income($),African_American(Count),African_American(%),...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White_(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,Autauga,Alabama,1001,1.0,55504,23.94,15.12,$58343.00,10687,19.25,...,0.10,1586,2.86,41336,74.47,51.34,22921.0,42.00,76.33,815.0
1,Baldwin,Alabama,1003,0.0,212628,21.85,19.95,$56607.00,19037,8.95,...,0.07,9675,4.55,176582,83.05,51.45,77060.0,42.28,78.60,2827.0
2,Barbour,Alabama,1005,1.0,25270,20.76,18.82,$32490.00,12115,47.94,...,0.19,1063,4.21,11613,45.96,47.23,18613.0,67.79,75.78,451.0
3,Bibb,Alabama,1007,0.0,22668,20.61,16.02,$45795.00,4864,21.46,...,0.11,598,2.64,16842,74.30,46.45,15663.0,68.35,73.93,445.0
4,Blount,Alabama,1009,0.0,58013,23.35,17.84,$48253.00,847,1.46,...,0.12,5549,9.57,50439,86.94,50.69,51562.0,89.95,74.60,1050.0


In [6]:
df_merged = df.merge(df2, how='left', on='FIPS')
print(df_merged.shape)
df_merged.head()

(3142, 34)


,Unnamed: 0,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Native_Hawaiian/Other_Pacific_Islander(%),Hispanic(Count),Hispanic(%),Non-Hispanic_White_(Count),Non-Hispanic_White(%),Female(%),Rural(Count),Rural(%),Life_Expectancy,Deaths(Count)
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,0.10,1586.0,2.86,41336.0,74.47,51.34,22921.0,42.00,76.33,815.0
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,0.07,9675.0,4.55,176582.0,83.05,51.45,77060.0,42.28,78.60,2827.0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,0.19,1063.0,4.21,11613.0,45.96,47.23,18613.0,67.79,75.78,451.0
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,0.11,598.0,2.64,16842.0,74.30,46.45,15663.0,68.35,73.93,445.0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,0.12,5549.0,9.57,50439.0,86.94,50.69,51562.0,89.95,74.60,1050.0


In [7]:
df_merged = df_merged.dropna()

In [8]:
df_merged['Income($)'] = df_merged['Income($)'].str.replace('$', '')
df_merged['Income($)'] = df_merged['Income($)'].astype('float') / 100

In [9]:
df_merged = df_merged.astype({'Lifespan_Rank': 'int32', 'Life_Quality_Rank': 'int32','Health_Behaviors_Rank':'int32',
'Clinical_Care_Rank':'int32','Social_Economic_Factors_Rank':'int32','Physical_Environment_Rank':'int32'})

In [10]:
df3 = pd.read_csv(f"../../{file_path}Ticks_Cases_Lat_Long.csv")
df3.head()

,Unnamed: 0,FIPS,County,State,Lat,Long,Ticks_With_Lyme,Cases2019
0,0,1001.0,Autauga County,Alabama,32.5349,-86.6427,0,1
1,1,1003.0,Baldwin County,Alabama,30.7275,-87.7226,0,0
2,2,1005.0,Barbour County,Alabama,31.8696,-85.3932,0,1
3,3,1007.0,Bibb County,Alabama,32.9986,-87.1265,0,0
4,4,1009.0,Blount County,Alabama,33.9809,-86.5674,0,0


In [11]:
mldf = df_merged.merge(df3, on='FIPS')
print(mldf.shape)
mldf.head()

(3013, 41)


,Unnamed: 0_x,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Rural(%),Life_Expectancy,Deaths(Count),Unnamed: 0_y,County,State,Lat,Long,Ticks_With_Lyme,Cases2019
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,42.00,76.33,815.0,0,Autauga County,Alabama,32.5349,-86.6427,0,1
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,42.28,78.60,2827.0,1,Baldwin County,Alabama,30.7275,-87.7226,0,0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,67.79,75.78,451.0,2,Barbour County,Alabama,31.8696,-85.3932,0,1
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,68.35,73.93,445.0,3,Bibb County,Alabama,32.9986,-87.1265,0,0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,89.95,74.60,1050.0,4,Blount County,Alabama,33.9809,-86.5674,0,0


In [12]:
# Check for null values
for col in mldf:
    print(f'{col} has {mldf[col].isnull().sum()} nulls')

Unnamed: 0_x has 0 nulls
FIPS has 0 nulls
State_x has 0 nulls
County_x has 0 nulls
Lifespan_Rank has 0 nulls
Life_Quality_Rank has 0 nulls
Health_Behaviors_Rank has 0 nulls
Clinical_Care_Rank has 0 nulls
Social_Economic_Factors_Rank has 0 nulls
Physical_Environment_Rank has 0 nulls
County_y has 0 nulls
State_y has 0 nulls
Lyme_Disease_Incidence_Reported has 0 nulls
Population has 0 nulls
%<18_Yrs_Old has 0 nulls
%65_Yrs_Old_and_over has 0 nulls
Income($) has 0 nulls
African_American(Count) has 0 nulls
African_American(%) has 0 nulls
American_Indian/Alaskan_Native(Count) has 0 nulls
American_Indian/Alaskan_Native(%) has 0 nulls
Asian(Count) has 0 nulls
Asian(%) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(Count) has 0 nulls
Native_Hawaiian/Other_Pacific_Islander(%) has 0 nulls
Hispanic(Count) has 0 nulls
Hispanic(%) has 0 nulls
Non-Hispanic_White_(Count) has 0 nulls
Non-Hispanic_White(%) has 0 nulls
Female(%) has 0 nulls
Rural(Count) has 0 nulls
Rural(%) has 0 nulls
Life_Expectanc

In [13]:
mldf.dtypes

Unnamed: 0_x                                       int64
FIPS                                               int64
State_x                                           object
County_x                                          object
Lifespan_Rank                                      int32
Life_Quality_Rank                                  int32
Health_Behaviors_Rank                              int32
Clinical_Care_Rank                                 int32
Social_Economic_Factors_Rank                       int32
Physical_Environment_Rank                          int32
County_y                                          object
State_y                                           object
Lyme_Disease_Incidence_Reported                  float64
Population                                       float64
%<18_Yrs_Old                                     float64
%65_Yrs_Old_and_over                             float64
Income($)                                        float64
African_American(Count)        

In [14]:
# Lyme disease incident reported bin key
ldir_key_df = pd.DataFrame({
    "bin_sizes":['0 per 1000','0 - 0.25 per 1000','0.25 - 1 per 1000', '3 + per 1000'],
    "bin_ids": [0,1,2,3]
})
ldir_key_df.head()

,bin_sizes,bin_ids
0,0 per 1000,0
1,0 - 0.25 per 1000,1
2,0.25 - 1 per 1000,2
3,3 + per 1000,3


In [15]:
def map_fxn (x):
    if x ==0:
        return 0
    elif x <= 0.25:
        return 1
    elif x <= 1:
        return 2
    else:
        return 3

In [16]:
mldf['Norm_Incidence'] = mldf['Lyme_Disease_Incidence_Reported']/mldf['Population']*1000

In [17]:
mldf['Norm_Incidence'].describe()

count    3013.000000
mean        0.130944
std         0.415456
min         0.000000
25%         0.000000
50%         0.000000
75%         0.035140
max         5.830611
Name: Norm_Incidence, dtype: float64

In [18]:
# Map value to mldf to match the key df
mldf['Norm_Incidence_Bin']= mldf['Norm_Incidence'].map(map_fxn)
mldf.head()

,Unnamed: 0_x,FIPS,State_x,County_x,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,...,Deaths(Count),Unnamed: 0_y,County,State,Lat,Long,Ticks_With_Lyme,Cases2019,Norm_Incidence,Norm_Incidence_Bin
0,1,1001,Alabama,Autauga,8,5,15,14,5,50,...,815.0,0,Autauga County,Alabama,32.5349,-86.6427,0,1,0.018017,1
1,2,1003,Alabama,Baldwin,3,4,3,4,3,62,...,2827.0,1,Baldwin County,Alabama,30.7275,-87.7226,0,0,0.000000,0
2,3,1005,Alabama,Barbour,14,48,53,30,61,32,...,451.0,2,Barbour County,Alabama,31.8696,-85.3932,0,1,0.039573,1
3,4,1007,Alabama,Bibb,47,24,38,16,38,31,...,445.0,3,Bibb County,Alabama,32.9986,-87.1265,0,0,0.000000,0
4,5,1009,Alabama,Blount,36,14,10,41,15,53,...,1050.0,4,Blount County,Alabama,33.9809,-86.5674,0,0,0.000000,0


In [19]:
mldf['Norm_Incidence_Bin'].value_counts()

0    1878
1     783
2     235
3     117
Name: Norm_Incidence_Bin, dtype: int64

In [20]:
mldf = mldf.drop(['Unnamed: 0_x','Unnamed: 0_y','County_y','State_y','State_x','County_x','African_American(Count)','American_Indian/Alaskan_Native(Count)',
           'Hispanic(Count)','Non-Hispanic_White_(Count)','Asian(Count)',
           'Native_Hawaiian/Other_Pacific_Islander(Count)','Rural(Count)','Lyme_Disease_Incidence_Reported','County_x','State_x','County', 'State','Cases2019'], axis=1)

In [21]:
list(mldf.columns)

['FIPS',
 'Lifespan_Rank',
 'Life_Quality_Rank',
 'Health_Behaviors_Rank',
 'Clinical_Care_Rank',
 'Social_Economic_Factors_Rank',
 'Physical_Environment_Rank',
 'Population',
 '%<18_Yrs_Old',
 '%65_Yrs_Old_and_over',
 'Income($)',
 'African_American(%)',
 'American_Indian/Alaskan_Native(%)',
 'Asian(%)',
 'Native_Hawaiian/Other_Pacific_Islander(%)',
 'Hispanic(%)',
 'Non-Hispanic_White(%)',
 'Female(%)',
 'Rural(%)',
 'Life_Expectancy',
 'Deaths(Count)',
 'Lat',
 'Long',
 'Ticks_With_Lyme',
 'Norm_Incidence',
 'Norm_Incidence_Bin']

In [22]:
mldf.head()

,FIPS,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Population,%<18_Yrs_Old,%65_Yrs_Old_and_over,...,Non-Hispanic_White(%),Female(%),Rural(%),Life_Expectancy,Deaths(Count),Lat,Long,Ticks_With_Lyme,Norm_Incidence,Norm_Incidence_Bin
0,1001,8,5,15,14,5,50,55504.0,23.94,15.12,...,74.47,51.34,42.00,76.33,815.0,32.5349,-86.6427,0,0.018017,1
1,1003,3,4,3,4,3,62,212628.0,21.85,19.95,...,83.05,51.45,42.28,78.60,2827.0,30.7275,-87.7226,0,0.000000,0
2,1005,14,48,53,30,61,32,25270.0,20.76,18.82,...,45.96,47.23,67.79,75.78,451.0,31.8696,-85.3932,0,0.039573,1
3,1007,47,24,38,16,38,31,22668.0,20.61,16.02,...,74.30,46.45,68.35,73.93,445.0,32.9986,-87.1265,0,0.000000,0
4,1009,36,14,10,41,15,53,58013.0,23.35,17.84,...,86.94,50.69,89.95,74.60,1050.0,33.9809,-86.5674,0,0.000000,0


In [23]:
# Export Merged DF as csv
mldf.to_csv(f"../../{file_path}/ML_Demographic_LymeCase_HealthRank.csv", index=False)

## Split the Data into Training and Testing

In [24]:
# Create our features
X = mldf.drop(axis=1, columns='Norm_Incidence_Bin')

# Create our target
y = mldf['Norm_Incidence_Bin']
y.value_counts()

0    1878
1     783
2     235
3     117
Name: Norm_Incidence_Bin, dtype: int64

In [25]:
X.describe()

,FIPS,Lifespan_Rank,Life_Quality_Rank,Health_Behaviors_Rank,Clinical_Care_Rank,Social_Economic_Factors_Rank,Physical_Environment_Rank,Population,%<18_Yrs_Old,%65_Yrs_Old_and_over,...,Hispanic(%),Non-Hispanic_White(%),Female(%),Rural(%),Life_Expectancy,Deaths(Count),Lat,Long,Ticks_With_Lyme,Norm_Incidence
count,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3.013000e+03,3013.000000,3013.000000,...,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000,3013.000000
mean,30143.607700,47.004647,47.315632,47.492200,47.562562,47.479920,47.714570,1.068290e+05,22.281802,18.745456,...,9.429346,76.536893,49.907839,58.239300,77.445058,1225.150680,38.366096,-91.954839,0.137073,0.130944
std,15053.344197,41.618732,41.552463,41.732051,41.510817,41.660967,41.896352,3.393090e+05,3.380324,4.509754,...,13.777920,19.928270,2.201147,30.700675,2.912101,3207.772879,5.190837,12.395843,0.343981,0.415456
min,1001.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.718000e+03,7.160000,4.770000,...,0.520000,2.760000,26.570000,0.000000,62.590000,20.000000,19.598700,-164.027200,0.000000,0.000000
25%,18157.000000,17.000000,17.000000,17.000000,17.000000,17.000000,17.000000,1.197600e+04,20.260000,15.850000,...,2.280000,64.870000,49.460000,33.710000,75.510000,189.000000,34.640500,-97.724600,0.000000,0.000000
50%,29107.000000,37.000000,38.000000,38.000000,38.000000,38.000000,38.000000,2.694700e+04,22.240000,18.410000,...,4.210000,83.850000,50.320000,58.910000,77.540000,417.000000,38.359600,-90.113800,0.000000,0.000000
75%,45017.000000,65.000000,65.000000,65.000000,65.000000,65.000000,65.000000,6.971300e+04,24.000000,21.180000,...,9.650000,92.540000,51.000000,83.870000,79.270000,1001.000000,41.724200,-83.493200,0.000000,0.035140
max,56045.000000,243.000000,244.000000,244.000000,243.000000,244.000000,244.000000,1.016351e+07,41.240000,56.940000,...,96.320000,97.920000,57.000000,100.000000,97.970000,83305.000000,69.312200,-67.628800,1.000000,5.830611


In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=1, stratify=y)
X_train.shape

# Feature scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.transform(X_test)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier.

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

## Random Forest

In [27]:
# Different model:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Create a random forest classifier model
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# Fit the model 
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

## Pickle the model

In [28]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(brf, open('../pickle_data/brfmodel.pkl', 'wb'))

In [29]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/brfmodel.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [30]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

Accuracy Score : 0.9961734693877551


In [31]:
matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(matrix)
cm_df

,0,1,2,3
0,470,0,0,0
1,0,193,3,0
2,0,0,59,0
3,0,0,0,29


In [32]:
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      1.00      1.00      1.00      1.00      1.00       470
          1       1.00      0.98      1.00      0.99      0.99      0.98       196
          2       0.95      1.00      1.00      0.98      1.00      1.00        59
          3       1.00      1.00      1.00      1.00      1.00      1.00        29

avg / total       1.00      1.00      1.00      1.00      1.00      1.00       754



In [33]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X.columns), reverse= True)

[(0.4479376904388069, 'Norm_Incidence'),
 (0.07312155022527027, 'Long'),
 (0.05149401713673821, 'Lat'),
 (0.038118020440941515, '%<18_Yrs_Old'),
 (0.033686713495919764, 'FIPS'),
 (0.03337060203127531, 'Non-Hispanic_White(%)'),
 (0.03326504267559197, 'Population'),
 (0.02875550569222594, 'Deaths(Count)'),
 (0.023776336543311676, 'African_American(%)'),
 (0.021380188865088714, 'Income($)'),
 (0.01937402689440424, 'Life_Expectancy'),
 (0.01927267266355201, 'American_Indian/Alaskan_Native(%)'),
 (0.01811585689748836, 'Rural(%)'),
 (0.017171423863487547, 'Asian(%)'),
 (0.016948088621297437, 'Hispanic(%)'),
 (0.01504694097037207, 'Physical_Environment_Rank'),
 (0.01465408341474304, '%65_Yrs_Old_and_over'),
 (0.01341955271385356, 'Clinical_Care_Rank'),
 (0.012807588309722194, 'Life_Quality_Rank'),
 (0.01242964559789009, 'Lifespan_Rank'),
 (0.012367538791152129, 'Female(%)'),
 (0.01204644264046228, 'Social_Economic_Factors_Rank'),
 (0.01098744875244303, 'Health_Behaviors_Rank'),
 (0.0109102979

## Easy Ensemble

In [34]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train) 

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [35]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(eec, open('../pickle_data/easyensemble.pkl', 'wb'))

In [36]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/easyensemble.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [37]:
# Calculated the balanced accuracy score
acc_score2 = balanced_accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score2}")

Accuracy Score : 0.75


In [38]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(matrix)
cm_df

,0,1,2,3
0,470,0,0,0
1,0,196,0,0
2,0,59,0,0
3,0,0,0,29


# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the `SMOTE` algorithm. 

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [39]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 1408, 1: 1408, 3: 1408, 2: 1408})

In [40]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1)
classifier.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

## Pickle the model

In [41]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(classifier, open('../pickle_data/naiveoversample.pkl', 'wb'))

In [42]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/naiveoversample.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [43]:
# Calculated the balanced accuracy score
print(balanced_accuracy_score(y_test, predictions))

0.9241301368402913


In [44]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3
0,442,28,0,0
1,16,175,5,0
2,0,2,55,2
3,0,0,2,27


In [45]:
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.94      0.94      0.95      0.94      0.89       470
          1       0.85      0.89      0.95      0.87      0.92      0.84       196
          2       0.89      0.93      0.99      0.91      0.96      0.92        59
          3       0.93      0.93      1.00      0.93      0.96      0.92        29

avg / total       0.93      0.93      0.95      0.93      0.94      0.88       754



### SMOTE Oversampling

In [46]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampled)

Counter({0: 1408, 1: 1408, 3: 1408, 2: 1408})

In [47]:
# Train the Logistic Regression model using the resampled data
classifier2 = LogisticRegression(random_state=1)
classifier2.fit(X_resampled, y_resampled)
# Create a random forest classifier model
# brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# # Fit the model 
# brf.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

## Pickle the model

In [48]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(classifier2, open('../pickle_data/smotemodel.pkl', 'wb'))

In [49]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/smotemodel.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [50]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3
0,447,23,0,0
1,20,171,5,0
2,0,4,53,2
3,0,0,0,29


In [51]:
from imblearn.metrics import classification_report_imbalanced 
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      0.95      0.93      0.95      0.94      0.89       470
          1       0.86      0.87      0.95      0.87      0.91      0.82       196
          2       0.91      0.90      0.99      0.91      0.94      0.88        59
          3       0.94      1.00      1.00      0.97      1.00      1.00        29

avg / total       0.93      0.93      0.94      0.93      0.94      0.87       754



In [52]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

print(balanced_accuracy_score(y_test, predictions))

0.9304544735312084


# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. 

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [53]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({0: 88, 1: 88, 2: 88, 3: 88})

In [54]:
# Create Logistic Regression model
classifier3 = LogisticRegression(random_state=1)
classifier3.fit(X_resampled, y_resampled)

# # Create a random forest classifier model
# brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# # Fit the model 
# brf.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [55]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(classifier3, open('../pickle_data/undersample.pkl', 'wb'))

In [56]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/undersample.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [57]:
# Calculated the balanced accuracy score
print(balanced_accuracy_score(y_test, predictions))

0.8359421318672339


In [58]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3
0,399,70,1,0
1,44,127,25,0
2,0,5,52,2
3,0,0,1,28


In [59]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.90      0.85      0.85      0.87      0.85      0.72       470
          1       0.63      0.65      0.87      0.64      0.75      0.55       196
          2       0.66      0.88      0.96      0.75      0.92      0.84        59
          3       0.93      0.97      1.00      0.95      0.98      0.96        29

avg / total       0.81      0.80      0.87      0.81      0.83      0.69       754



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. 

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [60]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({0: 913, 1: 783, 2: 985, 3: 1244})

In [61]:
# Train the Logistic Regression model using the resampled data
classifier4 = LogisticRegression(random_state=1)
classifier4.fit(X_resampled, y_resampled)

# # Create a random forest classifier model
# brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)

# # Fit the model 
# brf.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [62]:
# saves a Python object to a pickled file using a relative path from the ../data/ directory.
pickle.dump(classifier4, open('../pickle_data/SMOTEENN.pkl', 'wb'))

In [63]:
# Load the model as an object
pickled_model = pickle.load(open('../pickle_data/SMOTEENN.pkl', 'rb'))

# Make predictions using the testing data
predictions = pickled_model.predict(X_test)

In [64]:
# X_test.to_dict('records')[0]

In [65]:
# # Export Merged DF as csv
# X_test.to_csv(f"../../{file_path}/pickle_tests.csv", index=False)

In [66]:
# Calculated the balanced accuracy score
print(balanced_accuracy_score(y_test, predictions))

0.33176035793528275


In [67]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report 

matrix = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    matrix)
cm_df

,0,1,2,3
0,184,13,118,155
1,78,1,21,96
2,17,0,2,40
3,3,0,0,26


In [68]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.65      0.39      0.65      0.49      0.51      0.25       470
          1       0.07      0.01      0.98      0.01      0.07      0.00       196
          2       0.01      0.03      0.80      0.02      0.16      0.03        59
          3       0.08      0.90      0.60      0.15      0.73      0.55        29

avg / total       0.43      0.28      0.75      0.31      0.38      0.18       754

